# Imports

In [17]:
import random
import pandas as pd
import numpy as np
from copy import deepcopy

# Local Data

In [18]:
df = pd.read_csv("data/players.csv")
df = df.drop(columns=['Unnamed: 0']) # drop the index column

# Problem Configuration

In [19]:
TEAM_SIZE = 7
NUM_TEAMS = 5
BUDGET_LIMIT = 750
TEAM_STRUCTURE = {"GK": 1, "DEF": 2, "MID": 2, "FWD": 2}
POPULATION_SIZE = 10

# Building Classes

In [20]:
class Player:
    def __init__(self, name, position, skill, salary):
        self.name = name
        self.position = position
        self.skill = skill
        self.salary = salary

    @classmethod
    def from_dict(cls, data):
        return cls(
            name=data['Name'],
            position=data['Position'],
            skill=data['Skill'],
            salary=data['Salary (€M)']
        )

    def to_dict(self):
        return {
            'Name': self.name,
            'Position': self.position,
            'Skill': self.skill,
            'Salary (€M)': self.salary
        }

    def __repr__(self):
        return f"{self.position}: {self.name} | Skill: {self.skill} | Salary: €{self.salary}M"

In [21]:
class Team:
    def __init__(self, players):
        self.players = players  # List of Player objects

    def is_valid(self, structure, budget):
        if len(self.players) != sum(structure.values()):
            return False

        pos_counts = {pos: 0 for pos in structure}
        total_salary = 0
        names_seen = set()

        for player in self.players:
            if player.name in names_seen:
                return False
            names_seen.add(player.name)
            pos_counts[player.position] += 1
            total_salary += player.salary

        return pos_counts == structure and total_salary <= budget

    def avg_skill(self):
        return sum(p.skill for p in self.players) / len(self.players)

    def total_salary(self):
        return sum(p.salary for p in self.players)

    def player_names(self):
        return [p.name for p in self.players]

    def __repr__(self):
        return "\n".join([f"  - {p}" for p in self.players])

In [22]:
class LeagueIndividual:
    def __init__(self, players_by_position, team_structure, budget_limit, league=None):
        
        self.players_by_position = players_by_position
        self.team_structure = team_structure
        self.budget_limit = budget_limit

        self.league = league if league is not None else self._generate_league()
        self.fitness = self.evaluate_fitness()

    def _generate_league(self):
        league = []
        all_players = deepcopy(self.players_by_position)
        used_names = set()

        for _ in range(NUM_TEAMS):
            team_players = []

            for pos, count in self.team_structure.items():
                candidates = [p for p in all_players[pos] if p.name not in used_names]
                if len(candidates) < count:
                    return None  # Not enough players available

                selected = random.sample(candidates, count)
                team_players.extend(selected)
                used_names.update(p.name for p in selected)

            team = Team(team_players)
            if not team.is_valid(self.team_structure, self.budget_limit):
                return None

            league.append(team)

            # Remove used players from pool
            for pos in self.players_by_position:
                all_players[pos] = [p for p in all_players[pos] if p.name not in used_names]

        return league

    def evaluate_fitness(self):
        if self.league is None:
            return float('inf')

        avg_skills = []
        used_names = set()

        for team in self.league:
            if not team.is_valid(self.team_structure, self.budget_limit):
                return float('inf')

            for p in team.players:
                if p.name in used_names:
                    return float('inf')  # Duplicate player across teams
                used_names.add(p.name)

            avg_skills.append(team.avg_skill())

        return np.std(avg_skills)

    def __lt__(self, other):
        return self.fitness < other.fitness

    def __repr__(self):
        return f"<LeagueIndividual fitness={self.fitness:.4f}>"

# Convert DF to player objects

In [23]:
players_by_position = {
    pos: [Player.from_dict(row) for _, row in df[df['Position'] == pos].iterrows()]
    for pos in TEAM_STRUCTURE
}

# Generate Population

In [24]:
# === GENERATE POPULATION ===
def generate_initial_population(size, players_by_position, team_structure, budget_limit):
    population = []
    attempts = 0
    max_attempts = 1000 # avoid infinite loop if unable to generate valid leagues

    while len(population) < size and attempts < max_attempts:
        indiv = LeagueIndividual(players_by_position, team_structure, budget_limit)
        if indiv.league is not None:
            population.append(indiv)
        attempts += 1

    return population

population = generate_initial_population(
    POPULATION_SIZE,
    players_by_position,
    TEAM_STRUCTURE,
    BUDGET_LIMIT
)

# Check if Classes are working

In [25]:
# === EXAMPLE USAGE ===
individual = LeagueIndividual(players_by_position, TEAM_STRUCTURE, BUDGET_LIMIT)

# Print result
print("\n=== One League Example ===")
for i, team in enumerate(individual.league):
    print(f"\n🏆 Team {i + 1}")
    print(team)
    print(f"Avg Skill: {team.avg_skill():.2f} | Total Salary: €{team.total_salary()}M")

print(f"\nFitness: {individual.fitness:.4f}")


=== One League Example ===

🏆 Team 1
  - GK: Ryan Mitchell | Skill: 83 | Salary: €85M
  - DEF: Logan Brooks | Skill: 86 | Salary: €95M
  - DEF: Mason Reed | Skill: 82 | Salary: €75M
  - MID: Dylan Morgan | Skill: 91 | Salary: €115M
  - MID: Dominic Bell | Skill: 86 | Salary: €95M
  - FWD: Elijah Sanders | Skill: 93 | Salary: €140M
  - FWD: Zachary Nelson | Skill: 86 | Salary: €92M
Avg Skill: 86.71 | Total Salary: €697M

🏆 Team 2
  - GK: Blake Henderson | Skill: 87 | Salary: €95M
  - DEF: Lucas Bennett | Skill: 85 | Salary: €90M
  - DEF: Owen Parker | Skill: 88 | Salary: €100M
  - MID: Ashton Phillips | Skill: 90 | Salary: €110M
  - MID: Spencer Ward | Skill: 84 | Salary: €85M
  - FWD: Adrian Collins | Skill: 85 | Salary: €90M
  - FWD: Xavier Bryant | Skill: 90 | Salary: €120M
Avg Skill: 87.00 | Total Salary: €690M

🏆 Team 3
  - GK: Chris Thompson | Skill: 80 | Salary: €80M
  - DEF: Caleb Fisher | Skill: 84 | Salary: €85M
  - DEF: Daniel Foster | Skill: 90 | Salary: €110M
  - MID: Nath

In [26]:


# === PRINT POPULATION DETAILS ===
for idx, indiv in enumerate(population):
    print("\n" + "=" * 35)
    print(f"🏟️  League (Individual) {idx + 1}")
    print("=" * 35)

    for tidx, team in enumerate(indiv.league):
        print(f"\n  🏆 Team {tidx + 1}")
        print(team)
        print(f"    📊 Avg Skill: {team.avg_skill():.2f}")
        print(f"    💰 Total Salary: €{team.total_salary()}M")

    print(f"\n  ➤ League Fitness (Std Dev of team avg skill): {indiv.fitness:.4f}")



🏟️  League (Individual) 1

  🏆 Team 1
  - GK: Alex Carter | Skill: 85 | Salary: €90M
  - DEF: Maxwell Flores | Skill: 81 | Salary: €72M
  - DEF: Brayden Hughes | Skill: 87 | Salary: €100M
  - MID: Austin Torres | Skill: 82 | Salary: €80M
  - MID: Hunter Cooper | Skill: 83 | Salary: €85M
  - FWD: Zachary Nelson | Skill: 86 | Salary: €92M
  - FWD: Chase Murphy | Skill: 86 | Salary: €95M
    📊 Avg Skill: 84.29
    💰 Total Salary: €614M

  🏆 Team 2
  - GK: Chris Thompson | Skill: 80 | Salary: €80M
  - DEF: Ethan Howard | Skill: 80 | Salary: €70M
  - DEF: Logan Brooks | Skill: 86 | Salary: €95M
  - MID: Bentley Rivera | Skill: 88 | Salary: €100M
  - MID: Dominic Bell | Skill: 86 | Salary: €95M
  - FWD: Sebastian Perry | Skill: 95 | Salary: €150M
  - FWD: Xavier Bryant | Skill: 90 | Salary: €120M
    📊 Avg Skill: 86.43
    💰 Total Salary: €710M

  🏆 Team 3
  - GK: Ryan Mitchell | Skill: 83 | Salary: €85M
  - DEF: Caleb Fisher | Skill: 84 | Salary: €85M
  - DEF: Lucas Bennett | Skill: 85 | S

# CROSSOVER

## Crossover By Team

In [27]:
# TEAM-BASED CROSSOVER
def team_crossover(parent1: LeagueIndividual, parent2: LeagueIndividual) -> tuple:
    crossover_point = random.randint(1, NUM_TEAMS - 1)

    def build_child(p1: LeagueIndividual, p2: LeagueIndividual) -> LeagueIndividual:
        child_teams = []
        used_names = set()

        # Copy prefix teams from parent 1
        for team in p1.league[:crossover_point]:
            child_teams.append(team)
            used_names.update(player.name for player in team.players)

        # Add valid teams from parent 2 without duplicate players
        for team in p2.league:
            names = [player.name for player in team.players]
            if len(child_teams) < NUM_TEAMS and all(name not in used_names for name in names):
                child_teams.append(team)
                used_names.update(names)

        # If needed, generate new teams to complete the league
        while len(child_teams) < NUM_TEAMS:
            team_players = []
            for position, count in TEAM_STRUCTURE.items():
                available = [
                    p for p in players_by_position[position]
                    if p.name not in used_names
                ]
                if len(available) < count:
                    break  # not enough players to form a valid team
                selected = random.sample(available, count)
                team_players.extend(selected)

            new_team = Team(team_players)
            if new_team.is_valid(TEAM_STRUCTURE, BUDGET_LIMIT):
                child_teams.append(new_team)
                used_names.update(p.name for p in team_players)

        return LeagueIndividual(players_by_position, TEAM_STRUCTURE, BUDGET_LIMIT, league=child_teams)

    child1 = build_child(parent1, parent2)
    child2 = build_child(parent2, parent1)
    return child1, child2


### Test Team Crossover

In [28]:
# Generate two valid parents
parent1 = LeagueIndividual(players_by_position, TEAM_STRUCTURE, BUDGET_LIMIT)
while parent1.league is None:
    parent1 = LeagueIndividual(players_by_position, TEAM_STRUCTURE, BUDGET_LIMIT)

parent2 = LeagueIndividual(players_by_position, TEAM_STRUCTURE, BUDGET_LIMIT)
while parent2.league is None:
    parent2 = LeagueIndividual(players_by_position, TEAM_STRUCTURE, BUDGET_LIMIT)

child1, child2 = team_crossover(parent1, parent2)

def print_league(league_individual, label):
    print(f"\n{label} | Fitness: {league_individual.fitness:.4f}")
    for i, team in enumerate(league_individual.league):
        print(f"  Team {i+1}:")
        for p in team.players:
            print(f"    {p.name} ({p.position}) - Skill: {p.skill} - Salary: €{p.salary}M")

print_league(parent1, "Parent 1")
print_league(parent2, "Parent 2")
print_league(child1, "Child 1")
print_league(child2, "Child 2")



Parent 1 | Fitness: 1.2434
  Team 1:
    Chris Thompson (GK) - Skill: 80 - Salary: €80M
    Ethan Howard (DEF) - Skill: 80 - Salary: €70M
    Jaxon Griffin (DEF) - Skill: 79 - Salary: €65M
    Hunter Cooper (MID) - Skill: 83 - Salary: €85M
    Dominic Bell (MID) - Skill: 86 - Salary: €95M
    Adrian Collins (FWD) - Skill: 85 - Salary: €90M
    Sebastian Perry (FWD) - Skill: 95 - Salary: €150M
  Team 2:
    Blake Henderson (GK) - Skill: 87 - Salary: €95M
    Brayden Hughes (DEF) - Skill: 87 - Salary: €100M
    Logan Brooks (DEF) - Skill: 86 - Salary: €95M
    Dylan Morgan (MID) - Skill: 91 - Salary: €115M
    Austin Torres (MID) - Skill: 82 - Salary: €80M
    Elijah Sanders (FWD) - Skill: 93 - Salary: €140M
    Zachary Nelson (FWD) - Skill: 86 - Salary: €92M
  Team 3:
    Ryan Mitchell (GK) - Skill: 83 - Salary: €85M
    Mason Reed (DEF) - Skill: 82 - Salary: €75M
    Daniel Foster (DEF) - Skill: 90 - Salary: €110M
    Bentley Rivera (MID) - Skill: 88 - Salary: €100M
    Ashton Phillip

## Crossover By Position

In [29]:
# POSITION-BASED CROSSOVER
def position_crossover(parent1: LeagueIndividual, parent2: LeagueIndividual) -> tuple:
    # Step 1: Collect all players by position from both parents
    combined_by_position = {pos: [] for pos in TEAM_STRUCTURE}
    used_names = set()

    for individual in [parent1, parent2]:
        for team in individual.league:
            for player in team.players:
                if player.name not in used_names:
                    combined_by_position[player.position].append(player)
                    used_names.add(player.name)

    # Step 2: Shuffle each position group and split into two sets
    child1_pool = {pos: [] for pos in TEAM_STRUCTURE}
    child2_pool = {pos: [] for pos in TEAM_STRUCTURE}

    for pos, players in combined_by_position.items():
        random.shuffle(players)
        midpoint = len(players) // 2
        child1_pool[pos] = players[:midpoint]
        child2_pool[pos] = players[midpoint:]

    # Step 3: Fill up missing players in each pool if needed
    def fill_position_pool(pool):
        for pos, required_count in TEAM_STRUCTURE.items():
            total_needed = required_count * NUM_TEAMS
            current_count = len(pool[pos])
            available = [
                p for p in players_by_position[pos]
                if p.name not in {x.name for x in pool[pos]}
            ]
            if current_count < total_needed:
                extra = random.sample(available, total_needed - current_count)
                pool[pos].extend(extra)
        return pool

    child1_pool = fill_position_pool(child1_pool)
    child2_pool = fill_position_pool(child2_pool)

    # Step 4: Build teams from the position pools
    def build_league_from_pool(pool) -> LeagueIndividual:
        all_players = deepcopy(pool)
        league = []
        used_names = set()

        for _ in range(NUM_TEAMS):
            team_players = []

            for pos, count in TEAM_STRUCTURE.items():
                candidates = [p for p in all_players[pos] if p.name not in used_names]
                if len(candidates) < count:
                    break  # cannot build a valid team
                selected = random.sample(candidates, count)
                team_players.extend(selected)
                used_names.update(p.name for p in selected)
                all_players[pos] = [p for p in all_players[pos] if p.name not in used_names]

            team = Team(team_players)
            if not team.is_valid(TEAM_STRUCTURE, BUDGET_LIMIT):
                return LeagueIndividual(players_by_position, TEAM_STRUCTURE, BUDGET_LIMIT, league=None)

            league.append(team)

        return LeagueIndividual(players_by_position, TEAM_STRUCTURE, BUDGET_LIMIT, league=league)

    child1 = build_league_from_pool(child1_pool)
    child2 = build_league_from_pool(child2_pool)

    return child1, child2

### Test Position Crossover

In [30]:
# Generate two valid parents
parent1 = LeagueIndividual(players_by_position, TEAM_STRUCTURE, BUDGET_LIMIT)
while parent1.league is None:
    parent1 = LeagueIndividual(players_by_position, TEAM_STRUCTURE, BUDGET_LIMIT)

parent2 = LeagueIndividual(players_by_position, TEAM_STRUCTURE, BUDGET_LIMIT)
while parent2.league is None:
    parent2 = LeagueIndividual(players_by_position, TEAM_STRUCTURE, BUDGET_LIMIT)

# Apply position-based crossover
child1, child2 = position_crossover(parent1, parent2)

def print_league(individual, label):
    print(f"\n{label} | Fitness: {individual.fitness:.4f}")
    for i, team in enumerate(individual.league):
        print(f"  Team {i+1}:")
        for player in team.players:
            print(f"    {player.name} ({player.position}) - Skill: {player.skill} - Salary: €{player.salary}M")

print_league(parent1, "Parent 1")
print_league(parent2, "Parent 2")
print_league(child1, "Child 1")
print_league(child2, "Child 2")



Parent 1 | Fitness: 0.7793
  Team 1:
    Blake Henderson (GK) - Skill: 87 - Salary: €95M
    Owen Parker (DEF) - Skill: 88 - Salary: €100M
    Logan Brooks (DEF) - Skill: 86 - Salary: €95M
    Bentley Rivera (MID) - Skill: 88 - Salary: €100M
    Gavin Richardson (MID) - Skill: 87 - Salary: €95M
    Chase Murphy (FWD) - Skill: 86 - Salary: €95M
    Landon Powell (FWD) - Skill: 89 - Salary: €110M
  Team 2:
    Alex Carter (GK) - Skill: 85 - Salary: €90M
    Brayden Hughes (DEF) - Skill: 87 - Salary: €100M
    Lucas Bennett (DEF) - Skill: 85 - Salary: €90M
    Ashton Phillips (MID) - Skill: 90 - Salary: €110M
    Hunter Cooper (MID) - Skill: 83 - Salary: €85M
    Zachary Nelson (FWD) - Skill: 86 - Salary: €92M
    Sebastian Perry (FWD) - Skill: 95 - Salary: €150M
  Team 3:
    Jordan Smith (GK) - Skill: 88 - Salary: €100M
    Ethan Howard (DEF) - Skill: 80 - Salary: €70M
    Daniel Foster (DEF) - Skill: 90 - Salary: €110M
    Spencer Ward (MID) - Skill: 84 - Salary: €85M
    Austin Torre